This is a quick tutorial for running PowerFlowCVXRS.jl.
There are several packages that are used for the code, and the main ones are PowerModels, which is used to read test case data and solve OPF and power flow equations, and JuMP, which is used to solve resulting convex optimization problems using Mosek or Gurobi.

In [1]:
using JuMP, MosekTools, Gurobi
using SparseArrays, LinearAlgebra, Plots
using PowerModels, Ipopt
include("../src/main.jl")

# Import test case data
network_data = PowerModels.parse_file("../cases/case9.m");

ArgumentError: ArgumentError: Package MosekTools not found in current path.
- Run `import Pkg; Pkg.add("MosekTools")` to install the MosekTools package.

In [8]:
using CSV
using DataFrames
using JSON


In [15]:
pprintln(network_data["bus"])

Dict{String, Any}("8" => Dict{String, Any}("zone" => 1, "bus_i" => 8, "bus_type" => 1, "vmax" => 1.1, "source_id" => Any["bus", 8], "area" => 1, "vmin" => 0.9, "index" => 8, "va" => 0.0, "vm" => 1.0, "base_kv" => 345.0), "4" => Dict{String, Any}("zone" => 1, "bus_i" => 4, "bus_type" => 1, "vmax" => 1.1, "source_id" => Any["bus", 4], "area" => 1, "vmin" => 0.9, "index" => 4, "va" => 0.0, "vm" => 1.0, "base_kv" => 345.0), "1" => Dict{String, Any}("zone" => 1, "bus_i" => 1, "bus_type" => 3, "vmax" => 1.1, "source_id" => Any["bus", 1], "area" => 1, "vmin" => 0.9, "index" => 1, "va" => 0.0, "vm" => 1.0, "base_kv" => 345.0), "5" => Dict{String, Any}("zone" => 1, "bus_i" => 5, "bus_type" => 1, "vmax" => 1.1, "source_id" => Any["bus", 5], "area" => 1, "vmin" => 0.9, "index" => 5, "va" => 0.0, "vm" => 1.0, "base_kv" => 345.0), "2" => Dict{String, Any}("zone" => 1, "bus_i" => 2, "bus_type" => 2, "vmax" => 1.1, "source_id" => Any["bus", 2], "area" => 1, "vmin" => 0.9, "index" => 2, "va" => 0.0, "

The network data format uses parsing from PowerModels.jl.
Please refer to https://lanl-ansi.github.io/PowerModels.jl/stable/ for documentation.

There are two additional steps for running PowerFlowCVXRS. First, the initial dispatch point need to be solvable. In reality, using the data from measurements should satisfy this requirement, but for simulation here, we will run OPF with stricter constraints by calling "run opf_initialization."
Next, the feature for participation factor is added, and it needs to be set.

scrs runs sequential convex restriction to minimize the generation cost.

In [ ]:
## Initialize OPF
network_data=opf_initialization(network_data, 0.05)

## Set participation factor
[network_data["gen"][i]["ptc_factor"] = Int(network_data["bus"][string(gen["gen_bus"])]["bus_type"]==3) for (i,gen) in network_data["gen"]]

network_data=runpf(network_data)
println("Initial OPF objective solution: ",round(network_data["cost"],digits=2))

max_iter_SCRS = 10
network_data, result_cvxr = scrs(network_data, max_iter_SCRS); # run sequential convex restriction


Variables "result_cvxr["pg"]" and "result_cvxr["vg"]" contains a sequence of dispatch points, and a linear transition between consecutive points are guaranteed to provide a feasible path.

Same procedure can be used to ensure robustness of the solution. This requires a set up of "network_data["uncertainty"]," which specifies the uncertainty set.

In [ ]:
## Initialize OPF
network_data = opf_initialization(network_data, 0.0)

gamma_req=0.2
(Σ0,γ0)=network_data["uncertainty"]
network_data["uncertainty"]=(Σ0,gamma_req)

network_data, result_cvxr = scrs(network_data, max_iter_SCRS);

The solution in network_data is guaranteed to be robust against the uncertainty set described by the uncertainty set (Σ0,γ0).

Convex restriction can be visualized by creating a grid along two of the axis and solving for the slack variable. The following code provides a quick visualization.

For higher-quality visualization, check out https://github.com/dclee131/Visualizing-OPF.

In [ ]:
resolution = 20
plot_bus = ["1","2"] # Pick plot buses
plot_rng = [0 2 0 2] # Pick range for the plot [x1_min, x1_max, x2_min, x2_max]

U1_plot,U2_plot,exact_plot,cvxrs_plot,u_plot0 = plot2D(network_data,plot_bus,plot_rng,"pd",resolution) # Pass "pd" option to plot perturbation in loads

pyplot()
default(show = true)
contour(U1_plot,U2_plot,exact_plot,levels=0,color=:blues) # Plot feasible region
contour!(U1_plot,U2_plot,cvxrs_plot,levels=0,color=:greens) # Plot convex restriction
scatter!([u_plot0[1]],[u_plot0[2]],markersize=6, c=:red) # Plot nominal (base) point